In [ ]:
# Import necessary libaries
from pyspark.sql import SparkSession
from pyspark.sql.types import StructField, StructType, IntegerType, StringType
from datetime import datetime
import os

# Define the schema for dataframe
schema = StructType([
    StructField("domain", StringType()),
    StructField("pagename", StringType()),
    StructField("sumpageviewcount", IntegerType()),
    StructField("date_collection", StringType()),
    StructField("time_collection", StringType()),
    StructField("date_id", IntegerType()),
    StructField("time_id", IntegerType()),
    StructField("year", IntegerType()),
    StructField("month", IntegerType())
])

In [177]:
# create Spark from SparkSession 
spark = SparkSession.builder\
.appName("parquet_year_month_toDW")\
.master("local[4]")\
.config("spark.executor.cores", "4")\
.config("spark.executor.memory", "4g")\
.config("spark.jars","/opt/homebrew/Cellar/apache-spark/3.5.3/libexec/bin/postgresql-42.7.3.jar")\
.getOrCreate()

In [ ]:
# Define path to read and create dataframe
parquet_folder_path = "your_path"
df = spark.read.schema(schema).parquet(parquet_folder_path)
df.cache()

In [ ]:
# Define function to catch the activities
def log_write(message):
    time_now = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
    log_directory = os.path.dirname(logfile)
    if not os.path.exists(log_directory):
        os.makedirs(log_directory)
    with open(logfile, "a") as log:
        log.write(f"{time_now} : {message} \n")
    
# Define function read, transform dataframe
def create_df_to_write(reading_folder):
    df = spark.read.parquet(reading_folder)
    row_to_write = ["date_id","time_id", "domain", "pagename", "sumpageviewcount", "year", "month"]
    df_to_write = df.select(row_to_write)
    df_partitions = df_to_write.repartition('month')
    df_sorted= df_partitions.sortWithinPartitions('date_id', "time_id")
    return df_sorted

# Define function to insert data to data warehouse
def write_to_DW(reading_folder, table_name = 'fact_pageviews'):
    # Define df:
    df = create_df_to_write(reading_folder)
    # Define posgresql url
    postgresql_url = "jdbc:postgresql://localhost:5455/dwh_pageviews"
    # Define connection_properties
    connection_properties = {"user":"postgres",\
                         "driver":"org.postgresql.Driver",\
                            "batchsize":"10000"}
    # Write to database
    df.write.jdbc(url = postgresql_url,table = table_name, mode = 'append', properties = connection_properties)

In [ ]:

# Main logic for testing
if __name__ == '__main__':
    logfile = "your_path.txt"
    for file in range(2015, 2024):
        reading_path = "your_path"
        reading_folder = f'{reading_path}{file}/*'
        table_name = 'fact_pageviews'
        try: 
            log_write(f'writing {reading_path} to WD')
            write_to_DW(reading_folder,table_name)
            log_write(f'finish writing {reading_path} to WD')
        except Exception as e:
            log_write(f'got error as: {e}')

else: 
    print(None)

spark.stop()


